Below, I install my own implementation of Professor Boonstra's "memoize DataFrame to disk" feature. The source code can be found at [github.com/ethho/memoize](https://github.com/ethho/memoize).

In [1]:
!python3 -m pip install git+https://github.com/ethho/memoize.git

  Cloning https://github.com/ethho/memoize.git to /tmp/pip-req-build-lrz744ia
  Running command git clone --filter=blob:none --quiet https://github.com/ethho/memoize.git /tmp/pip-req-build-lrz744ia
  Resolved https://github.com/ethho/memoize.git to commit bef633bd22e4acde44cccb63399a176c6cef79b9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [21]:
import json
import re
import os
from glob import glob
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import quandl
import functools
import plotly.express as px
import plotly.graph_objects as go
from joblib import Parallel, delayed
import multiprocessing
from multiprocessing import Pool
from src.ubacktester import (
    BacktestEngine, StrategyBase, PositionBase, FeedBase,
    PlotlyPlotter, FeedID, PriceFeed, px_plot, ClockBase
)
from memoize.dataframe import memoize_df

%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format

# 20230202_hw4_ho_ethan_12350006

@mpcs
@finm33550

Ethan Ho 2/2/2023

----

## Configuration & Helper Functions

The following cell contains helper functions and configuration options that I will use in this notebook.

In [3]:
def get_secrets(fp='./secrets.json'):
    """
    Reads secret values such as API keys from a JSON-formatted file at `fp`.
    """
    with open(fp, 'r') as f:
        data = json.load(f)
    return data

def get_quandl_api_key() -> str:
    """
    Returns Quandl API key stored in secrets.json.
    """
    secrets = get_secrets()
    key = secrets.get('NASTAQ_DATA_API_KEY')
    assert key, f"NASTAQ_DATA_API_KEY field in secrets.json is empty or does not exist"
    return key

def strip_str_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a DataFrame, strips values in columns with string or object
    dtype. I noticed that this was an issue when I saw some m_ticker values
    like "AAPL       " with trailing whitespace.
    """
    for col in df.columns:
        if pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_object_dtype(df[col]):
            df[col] = df[col].str.strip()
    return df

@memoize_df(cache_dir='/tmp/memoize')
def fetch_quandl_quotemedia_prices(
    start_date, end_date, ticker
) -> pd.DataFrame:
    df = quandl.get_table(
        'QUOTEMEDIA/PRICES',
        date={'gte': start_date, 'lte': end_date},
        ticker=ticker,
        api_key=get_quandl_api_key(),
        paginate=True,
    )
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', inplace=True)
    return df

@memoize_df(cache_dir='/tmp/memoize')
def fetch_quandl_tbill_prices(
    start_date, end_date,
) -> pd.DataFrame:
    """Fetch table of treasury bill prices from Quandl."""
    df = quandl.get(
        ['USTREASURY/BILLRATES'],
        returns="pandas",
        start_date=start_date,
        end_date=end_date,
        ticker=ticker,
        api_key=get_quandl_api_key(),
    )
    df = df.reset_index().rename(columns={'Date': 'date'})
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', inplace=True)
    return df

def unique_index_keys(df, level=0) -> List[str]:
    return df.index.get_level_values(level=level).unique().tolist()

def risk_free_rate(**kw) -> float:
    """Calculates risk-free rate R_f from the 3-month T-bill rate."""
    tbill_prices = fetch_quandl_tbill_prices(**kw)
    tbill_returns = tbill_prices['USTREASURY/BILLRATES - 13 Wk Coupon Equiv']
    return tbill_returns.mean()

# Fetch High Frequency Trading Data

In [4]:
!ls data/Crypto/2021/For_Homework/

book_narrow_BTC-USD_2021.delim	trades_narrow_BTC-USD_2021.delim
book_narrow_ETH-BTC_2021.delim	trades_narrow_ETH-BTC_2021.delim
book_narrow_ETH-USD_2021.delim	trades_narrow_ETH-USD_2021.delim


In [6]:
DATA_DIR = 'data/Crypto'

def read_delim_gz(fp: str) -> pd.DataFrame:
    if fp.endswith('.gz'):
        !gzip -d {fp}
        fp = fp[:-3]
    # assert os.path.isfile(fp)
    df = pd.read_csv(fp, delim_whitespace=True)
    df.rename(columns={
        'timestamp_utc_nanoseconds': 'dt',
    }, inplace=True)
    return df

book = read_delim_gz('data/Crypto/2021/For_Homework/book_narrow_BTC-USD_2021.delim')
trades = read_delim_gz('data/Crypto/2021/For_Homework/trades_narrow_BTC-USD_2021.delim')
book.head(1000).tail()

,Ask1PriceMillionths,Bid1PriceMillionths,Ask1SizeBillionths,Bid1SizeBillionths,Ask2PriceMillionths,Bid2PriceMillionths,Ask2SizeBillionths,Bid2SizeBillionths,received_utc_nanoseconds,dt,Mid
995,61607370000,61607360000,180694490,45000000,61608500000,61607350000,280000000,46621560,1618561363576815100,1618561363622659000,"61,607,365,000.0000"
996,61607370000,61607360000,152106580,45000000,61608500000,61607350000,280000000,46621560,1618561363581388000,1618561363629171000,"61,607,365,000.0000"
997,61607370000,61607360000,219190760,45000000,61608500000,61607350000,280000000,46621560,1618561363585827000,1618561363633613000,"61,607,365,000.0000"
998,61607370000,61607360000,152106580,45000000,61608500000,61607350000,280000000,46621560,1618561363593017000,1618561363641559000,"61,607,365,000.0000"
999,61607370000,61607360000,180694490,45000000,61608500000,61607350000,280000000,46621560,1618561363603331000,1618561363649797000,"61,607,365,000.0000"


In [7]:
trades.head(1000).tail()

,received_utc_nanoseconds,dt,PriceMillionths,SizeBillionths,Side
995,1618090477003403000,1618090477019409000,59182390000,16837980,1.0000
996,1618090477101050000,1618090477110242000,59182390000,4152600,1.0000
997,1618090477418420000,1618090477437993000,59183600000,151460,1.0000
998,1618090478217125000,1618090478217639000,59183530000,1008600,1.0000
999,1618090479200375000,1618090479216013000,59181260000,151460,1.0000


Check that `Side` is integral for all values.

In [ ]:
(trades['Side'] == trades['Side'].astype(int)).all()

In [ ]:
2**63 / 9e18

In [27]:
@dataclass
class AccumulationStratBase(StrategyBase):
    USE_NS_DT = True
    
    def step(self):
        pass


@dataclass
class Trades(FeedBase):
    USE_NS_DT = True
    
    # timestamp_utc_nanoseconds: int
    name: str
    PriceMillionths: int = 0
    SizeBillionths: int = 0
    Side: int = 0
    
@dataclass
class Book(FeedBase):
    USE_NS_DT = True

    # timestamp_utc_nanoseconds: int
    Ask1PriceMillionths: int
    Bid1PriceMillionths: int
    Ask2PriceMillionths: int
    Bid2PriceMillionths: int
    Bid1SizeBillionths: int
    Ask1SizeBillionths: int
    Bid2SizeBillionths: int
    Ask2SizeBillionths: int

In [18]:
dti = pd.concat([book['dt'], trades['dt']]).sort_values().unique()

In [22]:
clock = ClockBase(dti)

In [28]:
trades_feed = Trades.from_df(trades)

In [ ]:
def test_accumulate():
    
    be = BacktestEngine(clock=clock)
    be.add_feed(trades_feed, name='trades')
    strat1 = AccumulationStratBase(cash_equity=1e4)
    be.add_strategy(strat1)
    be.run()

    # strat1.plot(
    #     show=False,
    #     # include_cols=['daily_pct_returns'],
    #     # scale_cols={'nshort': 40, 'nlong': 40}
    #     include_cols=['returns', 'nshort', 'nlong', ],
    #     scale_cols={'nshort': 40, 'nlong': 40, }
    # )

test_accumulate()